In [83]:
import os
import glob
import xml.etree.ElementTree as ET
data_path = os.path.join('data', 'motion_data', 'files_motions_589')

In [84]:
all_files = glob.glob(data_path+'/**/*[0-9].xml', recursive=True)
for files in all_files: 
    print(files)

data/motion_data/files_motions_589/pour02.xml
data/motion_data/files_motions_589/pour04.xml
data/motion_data/files_motions_589/pour03.xml
data/motion_data/files_motions_589/pour01.xml
data/motion_data/files_motions_589/pour05.xml
data/motion_data/files_motions_589/pour06.xml


In [157]:
motion_data_tree = ET.parse(all_files[0])
motion_demo_root = motion_data_tree.getroot()
print("Root: ",motion_demo_root.tag)

data_frame = []
joint_order = []
for child in motion_demo_root:
    print(child.tag, child.attrib)
    # For Motions
    for gchild in child:
        if gchild.tag=='JointOrder':
            for ggchild in gchild:
#                 print(">> ",ggchild.tag, ggchild.attrib)
                joint_order.append(ggchild.attrib['name'])
        if gchild.tag=='MotionFrames':
            for ggchild in gchild:
                if ggchild.tag=='MotionFrame':
#                     pass
                    try:
                        print("Timestep: ",ggchild[0].text)
                        print("Root Position: ",ggchild[1].text)
                        print("Root Rotation: ",ggchild[2].text)
                        print("Joint Positon: ", ggchild[3].text)
                    except IndexError:
                        pass

Root:  MMM
Motion {'name': 'subject_id_551'}
Timestep:  0
Root Position:  64.2596 95.7267 927.2
Root Rotation:  -0.0766947 -0.00392845 1.54134
Joint Positon:  -0.155053 -0.104687 -0.177841 0.0313283 0.0833478 -0.186167 0.0888019 -0.111069 0.203815 -0.0697384 0.0727235 -0.00533153 -0.0911834 -0.348357 0.120672 0.530735 -0.480835 0.0895263 0.0708034 0.0772251 -0.101181 -0.279311 0.444686 -0.184194 -0.123925 -0.509248 0.014595 0.160832 -0.0603089 0.349066 -0.17033 0.439349 0.333097 0.0729655 -0.0667705 -0.060373 -0.12589 -0.254338 -0.425918 0.234465 -0.0851316 0.49132 0.0157496 -0.194278 
Timestep:  0.01
Root Position:  64.2178 95.7837 927.252
Root Rotation:  -0.0768136 -0.0050163 1.5407
Joint Positon:  -0.161224 -0.101372 -0.184347 0.0323777 0.0819197 -0.185489 0.0883532 -0.10818 0.204823 -0.0654209 0.0686507 -0.00517856 -0.0888764 -0.348416 0.116489 0.527027 -0.463331 0.091593 0.0717291 0.0858286 -0.103847 -0.279718 0.445649 -0.191726 -0.13164 -0.50616 0.0136878 0.161665 -0.0639762 0.34

Timestep:  3.06
Root Position:  -637.671 33.3305 918.115
Root Rotation:  -0.119434 -0.0214748 1.48142
Joint Positon:  -0.339966 -0.109097 -0.249421 0.308347 -0.349066 0.15614 -0.368813 0.245854 0.245164 0.026846 0.020237 0.00364059 -0.00498196 -0.213885 -0.142757 0.401494 -0.299115 0.305468 0.0537611 0.339324 -0.346517 -0.049553 0.208255 -0.161297 -0.0740881 -0.284377 0.0539874 0.226931 -0.184633 0.349066 -0.459389 1.13677 1.1302 0.343858 -0.169405 -0.138266 -0.365849 -0.0672936 -0.363297 0.274125 -0.340974 -0.155786 0.0688729 -0.161668 
Timestep:  3.07
Root Position:  -637.851 33.2823 918.082
Root Rotation:  -0.119434 -0.0214748 1.48412
Joint Positon:  -0.339393 -0.109076 -0.24936 0.307158 -0.349066 0.158134 -0.368821 0.24577 0.243024 0.0266519 0.0178067 0.00279263 -0.00484936 -0.213879 -0.145418 0.402893 -0.295493 0.305361 0.0537289 0.339721 -0.346331 -0.0496792 0.209224 -0.163274 -0.0757678 -0.283684 0.0551093 0.22381 -0.184161 0.349066 -0.462344 1.14384 1.14946 0.343378 -0.169572 -

Root Position:  65.8473 111.323 928.415
Root Rotation:  -0.140567 -0.0457434 1.49417
Joint Positon:  -0.165371 -0.0784309 0.0179806 0.295939 -0.231814 -0.18918 -0.0259845 0.311536 0.309605 0.0321288 0.0141295 -0.343128 -0.0252956 -0.253199 -0.247225 0.447001 -0.392902 0.276685 0.186843 0.545466 -0.23981 -0.192746 0.370424 -0.358237 -0.121033 -0.436682 0.027066 0.167829 -0.125786 0.349066 -0.436977 0.545109 0.33183 0.0894809 0.0108635 0.213899 -0.0342447 -0.375025 -0.553597 0.214762 -0.101686 0.432907 0.0219776 -0.174943 
Timestep:  7.91
Root Position:  66.0114 111.412 928.425
Root Rotation:  -0.140391 -0.0455998 1.49436
Joint Positon:  -0.166257 -0.0794056 0.0185049 0.295768 -0.231803 -0.188396 -0.0259348 0.311137 0.309359 0.0329741 0.0152252 -0.343366 -0.0253779 -0.25324 -0.247359 0.447106 -0.397319 0.276843 0.186808 0.545379 -0.23994 -0.192918 0.370536 -0.358027 -0.121324 -0.4373 0.0253263 0.166607 -0.125897 0.349066 -0.438387 0.545264 0.331082 0.0896768 0.0109939 0.214768 -0.034593 

Timestep:  5.15
Root Position:  -978.678 543.875 723.329
Root Rotation:  0.0120822 0.00612687 -1.47507
Timestep:  5.16
Root Position:  -978.661 543.854 723.337
Root Rotation:  0.0120847 0.00586288 -1.47631
Timestep:  5.17
Root Position:  -978.603 543.835 723.356
Root Rotation:  0.0120899 0.00533188 -1.47708
Timestep:  5.18
Root Position:  -978.524 543.818 723.369
Root Rotation:  0.0119343 0.00463575 -1.47767
Timestep:  5.19
Root Position:  -978.54 543.805 723.384
Root Rotation:  0.010959 0.00373539 -1.47801
Timestep:  5.2
Root Position:  -978.552 543.805 723.399
Root Rotation:  0.0100987 0.00295873 -1.47844
Timestep:  5.21
Root Position:  -978.521 543.838 723.414
Root Rotation:  0.00989441 0.00259005 -1.47836
Timestep:  5.22
Root Position:  -978.528 543.858 723.426
Root Rotation:  0.00971127 0.00244307 -1.47817
Timestep:  5.23
Root Position:  -978.56 543.91 723.434
Root Rotation:  0.00970497 0.00281635 -1.47767
Timestep:  5.24
Root Position:  -978.596 543.941 723.44
Root Rotation:  0.0

Root Rotation:  0.0035992 -0.00561733 0.349584
Timestep:  1.16
Root Position:  -1005.22 73.0931 723.8
Root Rotation:  0.00359922 -0.00561728 0.349626
Timestep:  1.17
Root Position:  -1005.22 73.096 723.798
Root Rotation:  0.00359922 -0.00561734 0.349626
Timestep:  1.18
Root Position:  -1005.21 73.1012 723.799
Root Rotation:  0.00359905 -0.00565712 0.349734
Timestep:  1.19
Root Position:  -1005.21 73.1037 723.8
Root Rotation:  0.00359957 -0.00564667 0.349791
Timestep:  1.2
Root Position:  -1005.21 73.1077 723.801
Root Rotation:  0.00359954 -0.00565398 0.349882
Timestep:  1.21
Root Position:  -1005.21 73.1127 723.802
Root Rotation:  0.00359955 -0.005633 0.349996
Timestep:  1.22
Root Position:  -1005.21 73.1127 723.804
Root Rotation:  0.00359955 -0.00563301 0.349996
Timestep:  1.23
Root Position:  -1005.21 73.1136 723.806
Root Rotation:  0.00359956 -0.00565389 0.350048
Timestep:  1.24
Root Position:  -1005.21 73.113 723.807
Root Rotation:  0.00359955 -0.00565389 0.350048
Timestep:  1.25
R

Root Position:  -1005.24 73.1239 723.784
Root Rotation:  0.00445612 -0.00525027 0.349798
Timestep:  7.11
Root Position:  -1005.23 73.1277 723.784
Root Rotation:  0.00443827 -0.00525967 0.349975
Timestep:  7.12
Root Position:  -1005.23 73.1277 723.784
Root Rotation:  0.00442292 -0.00531445 0.34996
Timestep:  7.13
Root Position:  -1005.23 73.1263 723.781
Root Rotation:  0.00440497 -0.00533078 0.349962
Timestep:  7.14
Root Position:  -1005.23 73.1228 723.78
Root Rotation:  0.00440497 -0.00535165 0.349868
Timestep:  7.15
Root Position:  -1005.23 73.1196 723.78
Root Rotation:  0.00440497 -0.00535165 0.349868
Timestep:  7.16
Root Position:  -1005.23 73.1178 723.78
Root Rotation:  0.00440497 -0.00535165 0.349868
Timestep:  7.17
Root Position:  -1005.23 73.1178 723.78
Root Rotation:  0.00440497 -0.00535165 0.349868
Timestep:  7.18
Root Position:  -1005.23 73.1179 723.78
Root Rotation:  0.00440497 -0.00535161 0.349868
Timestep:  7.19
Root Position:  -1005.23 73.1191 723.78
Root Rotation:  0.004

Root Rotation:  0.00271781 0.00992852 -3.08817
Timestep:  5.15
Root Position:  -1272.43 154.196 -1.3981
Root Rotation:  0.00272119 0.00991141 -3.08818
Timestep:  5.16
Root Position:  -1272.43 154.194 -1.3981
Root Rotation:  0.00272286 0.00991141 -3.08818
Timestep:  5.17
Root Position:  -1272.43 154.194 -1.39643
Root Rotation:  0.00272485 0.00991141 -3.08818
Timestep:  5.18
Root Position:  -1272.43 154.197 -1.39143
Root Rotation:  0.00272652 0.00991144 -3.08818
Timestep:  5.19
Root Position:  -1272.43 154.194 -1.39073
Root Rotation:  0.00273207 0.0099128 -3.08819
Timestep:  5.2
Root Position:  -1272.43 154.175 -1.38896
Root Rotation:  0.00275268 0.00991326 -3.08819
Timestep:  5.21
Root Position:  -1272.42 154.177 -1.38771
Root Rotation:  0.00275421 0.00991326 -3.08819
Timestep:  5.22
Root Position:  -1272.42 154.172 -1.38792
Root Rotation:  0.00275585 0.00991326 -3.08819
Timestep:  5.23
Root Position:  -1272.42 154.17 -1.38944
Root Rotation:  0.00275695 0.00991312 -3.0882
Timestep:  5.2

In [158]:
print(joint_order)

['BLNx_joint', 'BLNy_joint', 'BLNz_joint', 'BPx_joint', 'BPy_joint', 'BPz_joint', 'BTx_joint', 'BTy_joint', 'BTz_joint', 'BUNx_joint', 'BUNy_joint', 'BUNz_joint', 'LAx_joint', 'LAy_joint', 'LAz_joint', 'LEx_joint', 'LEz_joint', 'LHx_joint', 'LHy_joint', 'LHz_joint', 'LKx_joint', 'LSx_joint', 'LSy_joint', 'LSz_joint', 'LWx_joint', 'LWy_joint', 'LFx_joint', 'LMrot_joint', 'RAx_joint', 'RAy_joint', 'RAz_joint', 'REx_joint', 'REz_joint', 'RHx_joint', 'RHy_joint', 'RHz_joint', 'RKx_joint', 'RSx_joint', 'RSy_joint', 'RSz_joint', 'RWx_joint', 'RWy_joint', 'RFx_joint', 'RMrot_joint']


In [105]:
motion_demo_root[0].tag=='Motion'

True

In [148]:
print(motion_demo_root[0].tag)
print(motion_demo_root[0][1].tag)
print(motion_demo_root[0][2].tag)
print(motion_demo_root[0][3].tag)

Motion
ModelProcessorConfig
JointOrder
MotionFrames


In [108]:
motion_demo_root[0][0][0].tag

'File'